In [0]:
import pandas as pd
data_final=pd.read_csv("/content/drive/My Drive/data_final(reddit)")

In [0]:
data_final

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,author,authors,body,comment,comms_num,created,flair,id,score,title,url
0,0,0.0,0.0,dhavalcoholic,ICICIPruLifeIns,reposting lack activity r askindiahello last y...,dear policy holder dhavalcoholic request help ...,1,1.386254e+09,AskIndia,1s57oi,1,need feedback insurance policy took xpost aski...,https://www.reddit.com/r/india/comments/1s57oi...
1,1,1.0,1.0,amitkumarthakur,RAD-Business RAD-Business None barcam10 _snor...,24hrs local police station register case dont ...,calm downgo sp office town file grievance imme...,24,1.554080e+09,AskIndia,b7pvwt,94,somebody want kill full family,https://www.reddit.com/r/india/comments/b7pvwt...
2,2,2.0,2.0,FrustratedOCIHopeful,plshelpthedog ayyylmaaaoo Proper_Boysenberry ...,hello askindia first time poster long time lur...,honestly supervisor behaved exactly government...,27,1.555361e+09,AskIndia,bdfid1,10,ambassador india takes back newly issued oci c...,https://www.reddit.com/r/india/comments/bdfid1...
3,3,3.0,3.0,aloo_vs_bhaloo,vcdarklord tilismilis aloo_vs_bhaloo dogaa fo...,r tooafraidtoask india edition,modi control sex desires jerk someone else pro...,22,1.566529e+09,AskIndia,cu1xn4,18,randians afraid ask,https://www.reddit.com/r/india/comments/cu1xn4...
4,4,4.0,4.0,multubunu,NaN,hello submitted r raskindia week ago got answe...,NaN,0,1.361085e+09,AskIndia,18ntue,0,askindia cingari cengar tzengar,https://www.reddit.com/r/india/comments/18ntue...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3555,2425,NaN,NaN,AlternativeDrop6,jnik27promiscuous_bhismaKemosahbeolololopololo...,NaN,Bc dunia khtm hogi is saal lagta haiThe resear...,35,1.586954e+09,Coronavirus,g1l2lv,138,Coronavirus Mutation Found in India “Raises Al...,https://weather.com/en-IN/india/coronavirus/ne...
3556,2426,NaN,NaN,Dumma1729,qwertyzxcvbnmpoiuy-The-Bat-budbukAnxiousBlocka...,NaN,Because as the great George Carlin once said -...,26,1.587671e+09,Coronavirus,g6l7hl,85,"As India’s Poor Demand Relief, Why Are Middle ...",https://www.newsclick.in/India-Poor-Demand-Rel...
3557,2427,NaN,NaN,GL4389,BhayanakMuutabd_min_ibadillahKemosahbeTimbaktu...,NaN,> None of them have shown any symptoms of Covi...,18,1.587410e+09,Coronavirus,g4qvfx,153,"Out of money, 6 foreigners make home inside ca...",https://www.hindustantimes.com/india-news/out-...
3558,2428,NaN,NaN,n1ght_w1ng08,vindicators_riseKissMyBBQMAA_KI_CHUDIYAmrfreez...,NaN,The other day I saw a video where business was...,36,1.586001e+09,Coronavirus,fumux8,573,Doctors say India must prepare for an 'onslaug...,https://edition.cnn.com/2020/04/03/asia/india-...


In [0]:
data_final=data_final.fillna("True")

**Importing necessary libraries**

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.externals import joblib
import pickle
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

Defining Flairs

In [0]:
flairs = ["AskIndia", "Non-Political", "[R]eddiquette", 
          "Scheduled", "Photography", "Science/Technology",
          "Politics", "Business/Finance", "Policy/Economy",
          "Sports", "Food", "AMA","Coronavirus"]

**Making funcitons for different ML algorithms to be used**

**Logistic_Regression**

In [0]:
def logisticreg(X_train, X_test, y_train, y_test):

  from sklearn.linear_model import LogisticRegression

  logreg = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', LogisticRegression(n_jobs=1, C=1e3)),
                 ])
  logreg.fit(X_train, y_train)

  y_pred = logreg.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  print(classification_report(y_test, y_pred,target_names=flairs))

**Naive_Bayes_Classifier**

In [0]:
def nb_classifier(X_train, X_test, y_train, y_test):
  nb = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('clf', MultinomialNB()),
                ])
  nb.fit(X_train, y_train)

  y_pred = nb.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  print(classification_report(y_test, y_pred,target_names=flairs))

**Linear_SVM**

In [0]:

def linear_svm(X_train, X_test, y_train, y_test):
  sgd = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
                 ])
  sgd.fit(X_train, y_train)

  y_pred = sgd.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  print(classification_report(y_test, y_pred,target_names=flairs))

**Random_Forest**

In [0]:
def randomforest(X_train, X_test, y_train, y_test):
  ranfor = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', RandomForestClassifier(n_estimators = 1000, random_state = 42)),
                 ])
  ranfor.fit(X_train, y_train)

  y_pred = ranfor.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  print(classification_report(y_test, y_pred,target_names=flairs))

**MLP_classifier**

In [0]:
def mlpclassifier(X_train, X_test, y_train, y_test):  
  mlp = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', MLPClassifier(hidden_layer_sizes=(30,30,30))),
                 ])
  mlp.fit(X_train, y_train)

  y_pred = mlp.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  print(classification_report(y_test, y_pred,target_names=flairs))

**This is the function which will take as input (features and labels) split them into training and testing (20%) and then call different machine learning algorithms (passing training and validation set as arguments)**

In [0]:
def train_test(X,y):
 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
   
    print("Results of Naive Bayes Classifier")
    nb_classifier(X_train, X_test, y_train, y_test)
    print('----------------------------------------------------------------------------------')
    print("Results of Linear Support Vector Machine")
    linear_svm(X_train, X_test, y_train, y_test)
    print('----------------------------------------------------------------------------------')
    print("Results of Logistic Regression")
    logisticreg(X_train, X_test, y_train, y_test)
    print('----------------------------------------------------------------------------------')
    print("Results of Random Forest")
    randomforest(X_train, X_test, y_train, y_test)
    print('----------------------------------------------------------------------------------')
    print("Results of MLP Classifier")
    mlpclassifier(X_train, X_test, y_train, y_test)
    
    

**Trying standalone imputs and their combination as features**
Training on comments and title alone gave significantly good models , whereas body was not present in some posts so it didn't performed well as a feature .

Combining these is a good idea to train model on more information per post .

In [0]:

flair = data_final.flair


c = data_final.comment
t = data_final.title
b = data_final.body
u = data_final.url

data_final['combined_features']=c+t+b+u

combined_features=data_final.combined_features


print("Flair Detection using Title as Feature")
train_test(t,flair)
print("Trying with different Feature -----------------------")
print("Flair Detection using Body as Feature")
train_test(b,flair)
print("Trying with different Feature -----------------------")
print("Flair Detection using URL as Feature")
train_test(u,flair)
print("Trying with different Feature -----------------------")
print("Flair Detection using Comments as Feature")
train_test(c,flair)
print("Trying with different Feature -----------------------")

print("Flair Detection using Combined Features")
train_test(combined_features,flair)

Flair Detection using Title as Feature
Results of Naive Bayes Classifier
accuracy 0.523876404494382
                    precision    recall  f1-score   support

          AskIndia       1.00      0.04      0.08        24
     Non-Political       0.34      0.81      0.48        59
     [R]eddiquette       0.45      0.31      0.37        68
         Scheduled       0.79      0.21      0.33        52
       Photography       0.00      0.00      0.00        21
Science/Technology       0.44      0.54      0.49        68
          Politics       0.81      0.75      0.78        68
  Business/Finance       0.32      0.60      0.41        57
    Policy/Economy       0.42      0.44      0.43        68
            Sports       0.86      0.79      0.82        91
              Food       0.57      0.42      0.48        69
               AMA       0.80      0.62      0.70        63
       Coronavirus       0.00      0.00      0.00         4

          accuracy                           0.52       71

**Clearly combined features is a better choice as per the classification report of different alorithms**

Saving the logistic regression model for deploying on website

In [0]:
#choosing logistic regression trained on combine features as final model

X_train, X_test, y_train, y_test = train_test_split(combined_features, flair, test_size=0.2, random_state = 42)

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg = logreg.fit(X_train, y_train)


pickle.dump(logreg,open("model_final(reddit).pkl",'wb'))





**Testing the saved model**

Installing and importing all the dependencies

In [0]:

!pip install praw
import praw
from praw.models import MoreComments
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords




Authentication part

In [0]:
reddit = praw.Reddit(client_id = "####",
                     client_secret = "####",
                     user_agent = "####",
                     username = "####",
                     password = "####")

subreddit = reddit.subreddit('india')

Downloading nltk corpus

In [0]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**Defining the clean text function to apply cleaning on the data scrapped from the test url before passing it in the model**

In [0]:
replace_by_space = re.compile('[/(){}\[\]\|@,;]')
replace_symbol = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = replace_by_space.sub(' ', text) # replace certain symbols by space in text
    text = replace_symbol.sub('', text) # delete symbols from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove STOPWORDS from text
    return text

**loading the saved model**

**Given the url , this functions scrape data from that url which includes title,body,url,top 15 comments and then calls the saved model for prediction**

In [0]:
#testing


with open('/content/model_final(reddit).pkl', 'rb') as f:
    loaded_model = pickle.load(f)


def detect_flair(url):

  submission = reddit.submission(url=url)

  data = {}

  data['title'] = submission.title
  data['url'] = submission.url
  data['body'] = submission.body

  submission.comments.replace_more(limit=None)
  comment = ''
  chk=0
  for top_level_comment in submission.comments:
    comment = comment + ' ' + top_level_comment.body
    chk=chk+1
    if(chk>15):
      break
  data["comment"] = comment
  data['title'] = clean_text(data['title'])
  data['comment'] = clean_text(data['comment'])
  data['body'] = clean_text(data['body'])
  data['url'] = clean_text(data['url'])

  data['combine'] = data['title'] + data['comment'] + data['url']+data['body']
  
  return loaded_model.predict([data['combine']])

**Sample prediction**

In [0]:
detect_flair("https://www.reddit.com/r/india/comments/g88vql/govt_discomfort_with_rel_jio_tower_deal_over_fdi/")

array(['Business/Finance'], dtype=object)